In [1]:
!pip install transformers
!pip install numpy==1.26.0
!pip install torch torchvision torchaudio
!pip install --upgrade ipywidgets
!pip install pandas
!pip install torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 92.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/2

In [2]:
!nvidia-smi

Wed Mar 12 01:02:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   36C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!git clone https://github.com/avyas21/interpretablellm.git

Cloning into 'interpretablellm'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 73 (delta 30), reused 41 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (73/73), 6.48 MiB | 12.07 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [4]:
%cd interpretablellm
!ls

/content/interpretablellm
baseline.ipynb	baseline_prompt.txt  data  README.md  setup_dataset.ipynb


## Data Preprocessing

In [5]:
import pandas as pd

In [6]:
train_data = pd.read_csv("data/train_data.csv")
test_data = pd.read_csv("data/test_data.csv")

In [7]:
POSITIVE_WORDS = ["positive", "great", "good", "happy", "amazing", "fantastic", "yes"]
NEGATIVE_WORDS = ["negative", "bad", "sad", "terrible", "horrible", "no", "critical"]

In [8]:
def convert_lbl_to_int(label):
    if label.lower() in POSITIVE_WORDS:
        return 1
    if label.lower() in NEGATIVE_WORDS:
        return 0
    return -1

## Baseline Model

In [9]:
import numpy as np
print(np.__version__)
from transformers import BertModel, BertTokenizer, BertForSequenceClassification, BertConfig
from transformers import pipeline
from sklearn.metrics import f1_score
from torchinfo import summary
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import random

1.26.4


In [10]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label)
        }

In [11]:
def get_inputs_labels(df):
    inputs = []
    labels = []
    for idx, row in train_data.iterrows():
        input_text = row['Review']
        inputs.append(input_text)
        labels.append(1 if row['Sentiment'] == 'positive' else 0)
    return inputs, labels


In [12]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
inputs, labels = get_inputs_labels(train_data)
dataset = TextDataset(inputs, labels, tokenizer, 512)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Baseline Model

In [13]:
def get_baseline_model():
  gpu_available = torch.cuda.is_available()
  model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
  if gpu_available:
    return model.to(torch.device("cuda"))
  return model

In [14]:
baseline_model = get_baseline_model()
summary(baseline_model)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Layer (type:depth-idx)                                       Param #
BertForSequenceClassification                                --
├─BertModel: 1-1                                             --
│    └─BertEmbeddings: 2-1                                   --
│    │    └─Embedding: 3-1                                   23,440,896
│    │    └─Embedding: 3-2                                   393,216
│    │    └─Embedding: 3-3                                   1,536
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─BertEncoder: 2-2                                      --
│    │    └─ModuleList: 3-6                                  85,054,464
│    └─BertPooler: 2-3                                       --
│    │    └─Linear: 3-7                                      590,592
│    │    └─Tanh: 3-8                                        --
├─Dropout: 1-2                                               --
├─L

In [15]:
def baseline_predict(model, df):
    model.eval()
    predictions = []
    labels = []
    gpu_available = torch.cuda.is_available()
    with torch.no_grad():
        for idx, row in df.iterrows():
            input_text = row['Review']
            encoding = tokenizer(input_text, add_special_tokens=True, max_length = 512, padding='max_length', truncation=True, return_tensors='pt')
            if gpu_available:
              input_ids = encoding['input_ids'].cuda()
              attention_mask = encoding['attention_mask'].cuda()
              output = model(encoding['input_ids'].cuda(), encoding['attention_mask'].cuda())
            else:
              input_ids = encoding['input_ids']
              attention_mask = encoding['attention_mask']
              output = model(encoding['input_ids'], encoding['attention_mask'])

            _, prediction = torch.max(output.logits, dim=1)
            predictions.append(prediction.item())
            labels.append(convert_lbl_to_int(row['Sentiment']))

    return predictions, labels

In [16]:
def baseline_score_model(model, train_df, test_df):
    train_f1 = None
    test_f1 = None

    if train_df is not None:
        train_predictions, train_labels = baseline_predict(model, train_df)
        train_f1 = f1_score(train_labels, train_predictions, average='micro')

    if test_df is not None:
        test_predictions, test_labels = baseline_predict(model, test_df)
        test_f1 = f1_score(test_labels, test_predictions, average='micro')

    return train_f1, test_f1


In [17]:
#import gc
def baseline_train_test(model, epochs, dataloader, train_df, test_df, learning_rate = 1e-3, calc_train_f1 = True):
    #torch.cuda.empty_cache()
    #gc.collect()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    i = 0

    for epoch in range(epochs):
        model.train()
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            i += 1
            if i % 10 == 0:
                print(loss)

        if calc_train_f1:
          train_f1, _ = baseline_score_model(model, train_df, None)
          print("Epoch: " + str(epoch) + " F1: " + str(train_f1) + " LOSS: " + str(loss))
    _, test_f1 = baseline_score_model(model, None, test_df)
    print("TEST F1: " + str(test_f1))

In [18]:
baseline_train_test(baseline_model, 1, dataloader, train_data, test_data, learning_rate = 1e-4)

tensor(0.4828, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4516, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3084, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2702, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6143, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3650, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3676, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2077, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1872, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2655, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2755, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2075, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1253, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1327, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2705, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4371, device='cuda:0', grad_fn=

## Model Utilities

In [39]:
def predict(model, df):
    model.eval()
    predictions = []
    labels = []
    gpu_available = torch.cuda.is_available()
    with torch.no_grad():
        for idx, row in df.iterrows():
            input_text = row['Review']
            encoding = tokenizer(input_text, add_special_tokens=True, max_length = 512, padding='max_length', truncation=True, return_tensors='pt')
            if gpu_available:
              input_ids = encoding['input_ids'].cuda()
              attention_mask = encoding['attention_mask'].cuda()
              prediction = model(encoding['input_ids'].cuda(), encoding['attention_mask'].cuda())
            else:
              input_ids = encoding['input_ids']
              attention_mask = encoding['attention_mask']
              prediction = model(encoding['input_ids'], encoding['attention_mask'])
            predictions.append(1 if prediction > 0.5 else 0)
            labels.append(convert_lbl_to_int(row['Sentiment']))

    return predictions, labels

In [21]:
def score_model(model, train_df, test_df):
    train_f1 = None
    test_f1 = None

    if train_df is not None:
        train_predictions, train_labels = predict(model, train_df)
        train_f1 = f1_score(train_labels, train_predictions, average='micro')

    if test_df is not None:
        test_predictions, test_labels = predict(model, test_df)
        test_f1 = f1_score(test_labels, test_predictions, average='micro')

    return train_f1, test_f1

In [47]:
def qualitative_analysis_model(baseline_model, final_model, train_df, test_df):
    train_f1 = None
    test_f1 = None

    if test_df is not None:
        test_predictions_base, test_labels_base = baseline_predict(baseline_model, test_df)
        test_f1_base = f1_score(test_labels_base, test_predictions_base, average='micro')
        test_predictions_final, test_labels_final = predict(final_model, test_df)
        test_f1_final = f1_score(test_labels_final, test_predictions_final, average='micro')

        # Find mismatches and print corresponding 'Review' column with model performance
        mismatches = []
        for pred_base, pred_final, review, label in zip(test_predictions_base, test_predictions_final, test_df['Review'], test_labels_final):
            if pred_base != pred_final:
                correct_model = "Baseline" if pred_base == label else "Final"
                mismatches.append(f"Review: {review} | Correct Model: {correct_model}")

        if mismatches:
            print("Mismatched Reviews in Test Data:")
            for mismatch in mismatches:
                print(mismatch)

    return train_f1, test_f1

In [23]:
def train_model(model, epochs, dataloader, train_df, learning_rate = 1e-3, calc_train_f1 = True):
    criterion = nn.BCELoss() ## If required define your own criterion
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = learning_rate)
    gpu_available = torch.cuda.is_available()

    for epoch in range(epochs):
        i = 0
        for batch in dataloader:
            targets = np.array(batch['label'])
            targets = torch.tensor(np.expand_dims(targets,axis=1)).float()
            optimizer.zero_grad()
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']

            if gpu_available:
              targets = targets.cuda()
              input_ids = input_ids.cuda()
              attention_mask = attention_mask.cuda()

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, targets)
            loss.backward()
            if i % 20 == 0:
              print(loss)
            optimizer.step()
            i += 1

        if calc_train_f1:
          train_f1, _ = score_model(model, train_df, None)
          print("Epoch: " + str(epoch) + " F1: " + str(train_f1) + " LOSS: " + str(loss))

## Probe Models

In [ ]:
class CustomBERTModel(nn.Module):
    def __init__(self, n, randomized=False, linear_1 = None, linear_2 = None, requires_grad = False):
        super(CustomBERTModel, self).__init__()
        if randomized:
          config = BertConfig()
          self.bert = BertModel(config).to(torch.device("cuda"))
        else:
          self.bert=  BertModel.from_pretrained("bert-base-uncased")

        self.bert.encoder.layer = nn.ModuleList(self.bert.encoder.layer[:n])

        if randomized:
            # Reinitialize weights for each layer in the encoder
            for layer in self.bert.encoder.layer:
                for module in layer.modules():
                  for mod in module.modules():
                      if isinstance(mod, (nn.Linear, nn.Conv2d)):
                          nn.init.xavier_uniform_(mod.weight)
                          if mod.bias is not None:
                              nn.init.zeros_(mod.bias)
                      elif isinstance(mod, nn.LayerNorm):
                          nn.init.ones_(mod.weight)
                          nn.init.zeros_(mod.bias)

        for param in self.bert.parameters():
            param.requires_grad = requires_grad

        self.dropout = nn.Dropout(0.5)

        ### New layers:

        if linear_1 is not None:
          self.linear_1 = linear_1
        else:
          self.linear_1 = nn.Linear(768, 256)

        if linear_2 is not None:
          self.linear_2 = linear_2
        else:
          self.linear_2 = nn.Linear(256, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, ids, mask):
        output = self.bert(input_ids=ids, attention_mask=mask, output_hidden_states=True)
        linear1_output = self.linear_1(self.dropout(output.last_hidden_state[:,0,:]))
        linear2_output = self.linear_2(self.dropout(linear1_output))
        sigmoid_output = self.sigmoid(linear2_output)
        return sigmoid_output


In [ ]:
def get_custom_bert_model(num_bert_layers, randomized_weights = False, linear_1 = None, linear_2 = None, requires_grad = False):
  gpu_available = torch.cuda.is_available()
  model = CustomBERTModel(num_bert_layers, randomized_weights, linear_1, linear_2, requires_grad)
  if gpu_available:
    return model.to(torch.device("cuda"))
  return model

## Baseline interpretability

In [ ]:
baseline_model = get_custom_bert_model(2, True)

In [ ]:
def baseline_interpretability(dataloader, train_df, test_df, min_n = 1, max_n = 12):
    probe_model_12 = get_custom_bert_model(12)
    train_model(probe_model_12, 10, dataloader, train_df, 1e-4)
    linear_1 = probe_model_12.linear_1
    linear_2 = probe_model_12.linear_2

    for n in range(min_n, max_n + 1):
        print("N: " + str(n))
        model = get_custom_bert_model(n, False, linear_1, linear_2)
        train_f1, test_f1 = score_model(model, train_df, test_df)
        print("TRAIN F1: " + str(train_f1) + " TEST F1: " + str(test_f1))


In [ ]:
baseline_interpretability(dataloader, train_data, test_data)

## Baseline Interpretability - Randomized weights

In [ ]:
def score_all_randomized_models(dataloader, train_df, test_df, epochs, max_n, learning_rate = 1e-3):
    model_scores = []
    for n in range(1,max_n + 1):
        print("N: " + str(n))
        model = get_custom_bert_model(n, True)
        train_model(model, epochs, dataloader, train_df, learning_rate)
        _, test_f1 = score_model(model, None, test_df)
        print("TEST F1: " + str(test_f1))
        model_scores.append([n, test_f1])

    return model_scores

In [ ]:
model_scores = score_all_randomized_models(dataloader, train_data, test_data, 10, 12)

## Probes

In [ ]:
def score_all_probe_models(dataloader, train_df, test_df, epochs, min_n, max_n, learning_rate = 1e-4):
    model_scores = []
    for n in range(min_n,max_n + 1):
        print("N: " + str(n))
        model = get_custom_bert_model(n)
        train_model(model, epochs, dataloader, train_df, learning_rate)
        _, test_f1 = score_model(model, None, test_df)
        print("TEST F1: " + str(test_f1))
        model_scores.append([n, test_f1])

    return model_scores

In [ ]:
score_all_probe_models(dataloader, train_data, test_data, 10, 1, 11, 1e-3)

## Scalar Mixing Weights

In [27]:
class ScalarMixingWeightModel(nn.Module):
    def __init__(self, n, i):
        super(ScalarMixingWeightModel, self).__init__()

        self.bert= BertModel.from_pretrained("bert-base-uncased")
        self.bert.encoder.layer = nn.ModuleList(self.bert.encoder.layer[:n])
        self.n = n
        self.layer_weights = nn.Parameter(torch.ones(self.n))
        self.softmax = nn.Softmax(dim=0)
        self.i = i

        self.gamma = nn.Parameter(torch.ones(1))
        self.dropout = nn.Dropout(0.5)

        for param in self.bert.parameters():
            param.requires_grad = False

        ### New layers:
        if self.i == 1:
          self.linear1 = nn.Linear(768, 1)
        elif self.i == 2:
          self.linear1 = nn.Linear(768, 384)
          self.linear2 = nn.Linear(384, 1)
        elif self.i == 3:
          self.linear1 = nn.Linear(768, 384)
          self.linear2 = nn.Linear(384, 192)
          self.linear3 = nn.Linear(192, 1)
        elif self.i == 4:
          self.linear1 = nn.Linear(768, 384)
          self.linear2 = nn.Linear(384, 192)
          self.linear3 = nn.Linear(192, 96)
          self.linear4 = nn.Linear(96, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, ids, mask):
        outputs = self.bert(input_ids=ids, attention_mask=mask, output_hidden_states=True)

        hidden_states = outputs.hidden_states[1:1 + self.n]

        normalized_weights = self.softmax(self.layer_weights)
        scalar_mixing_weight = self.gamma * sum(normalized_weights[i] * hidden_states[i] for i in range(self.n))

        if self.i == 1:
          linear1_output = self.dropout(self.linear1(scalar_mixing_weight[:, 0, :]))
          sigmoid_output = self.sigmoid(linear1_output)
        elif self.i == 2:
          linear1_output = self.linear1(scalar_mixing_weight[:, 0, :])
          linear2_output = self.linear2(self.dropout(linear1_output))
          sigmoid_output = self.sigmoid(linear2_output)
        elif self.i == 3:
          linear1_output = self.linear1(scalar_mixing_weight[:, 0, :])
          linear2_output = self.linear2(self.dropout(linear1_output))
          linear3_output = self.linear3(self.dropout(linear2_output))
          sigmoid_output = self.sigmoid(linear3_output)
        elif self.i == 4:
          linear1_output = self.linear1(scalar_mixing_weight[:, 0, :])
          linear2_output = self.linear2(self.dropout(linear1_output))
          linear3_output = self.linear3(self.dropout(linear2_output))
          linear4_output = self.linear4(self.dropout(linear3_output))
          sigmoid_output = self.sigmoid(linear4_output)

        return sigmoid_output


In [28]:
def get_scalar_mixing_model(num_bert_layers, i):
  gpu_available = torch.cuda.is_available()
  model = ScalarMixingWeightModel(num_bert_layers, i)

  if gpu_available:
    return model.to(torch.device("cuda"))

  return model

In [29]:
weights = {}
for i in range(11,13):
  test_scalar = get_scalar_mixing_model(i,1)
  train_model(test_scalar, 10, dataloader, train_data, learning_rate = 0.1)
  print(i, ":", test_scalar.layer_weights)
  weights[i] = test_scalar.layer_weights

KeyboardInterrupt: 

In [ ]:
lrs = [0.1, 0.01, 0.001, 0.0001]
for lr in lrs:
  for i in range(1,5):
    test_scalar = get_scalar_mixing_model(2, i)
    train_model(test_scalar, 10, dataloader, train_data, learning_rate = lr)
    print("Learning rate:", lr, "Number of layers", i)

## Final Model

In [24]:
class FinalModel(nn.Module):
    def __init__(self):
        super(FinalModel, self).__init__()
        self.bert =  BertModel.from_pretrained("bert-base-uncased")
        self.bert.encoder.layer = nn.ModuleList(self.bert.encoder.layer[:6])

        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        ### New layers:
        self.linear_1 = nn.Linear(768, 256)
        self.linear_2 = nn.Linear(256, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, ids, mask):
        output = self.bert(input_ids=ids, attention_mask=mask, output_hidden_states=True)

        linear1_output = self.linear_1(self.dropout(output[1]))
        linear1_output = self.relu(linear1_output)
        linear2_output = self.linear_2(self.dropout(linear1_output))
        sigmoid_output = self.sigmoid(linear2_output)
        return sigmoid_output


In [25]:
def get_final_model():
  gpu_available = torch.cuda.is_available()
  model = FinalModel()
  if gpu_available:
    return model.to(torch.device("cuda"))
  return model

In [26]:
import torch.nn as nn
import torch
from transformers import AutoTokenizer, BertModel
import pandas as pd

class ScalarMixingWeightModel(nn.Module):
    def __init__(self, n, i):
        super(ScalarMixingWeightModel, self).__init__()

        self.bert= BertModel.from_pretrained("bert-base-uncased")
        self.bert.encoder.layer = nn.ModuleList(self.bert.encoder.layer[:n])
        self.n = n
        self.layer_weights = nn.Parameter(torch.ones(self.n))
        self.softmax = nn.Softmax(dim=0)
        self.i = i

        self.gamma = nn.Parameter(torch.ones(1))
        self.dropout = nn.Dropout(0.5)

        for param in self.bert.parameters():
            param.requires_grad = False

        ### New layers:
        if self.i == 1:
          self.linear1 = nn.Linear(768, 1)
        elif self.i == 2:
          self.linear1 = nn.Linear(768, 384)
          self.linear2 = nn.Linear(384, 1)
        elif self.i == 3:
          self.linear1 = nn.Linear(768, 384)
          self.linear2 = nn.Linear(384, 192)
          self.linear3 = nn.Linear(192, 1)
        elif self.i == 4:
          self.linear1 = nn.Linear(768, 384)
          self.linear2 = nn.Linear(384, 192)
          self.linear3 = nn.Linear(192, 96)
          self.linear4 = nn.Linear(96, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, ids, mask):
        outputs = self.bert(input_ids=ids, attention_mask=mask, output_hidden_states=True)

        hidden_states = outputs.hidden_states[1:1 + self.n]

        normalized_weights = self.softmax(self.layer_weights)
        scalar_mixing_weight = self.gamma * sum(normalized_weights[i] * hidden_states[i] for i in range(self.n))

        if self.i == 1:
          linear1_output = self.dropout(self.linear1(scalar_mixing_weight[:, 0, :]))
          sigmoid_output = self.sigmoid(linear1_output)
        elif self.i == 2:
          linear1_output = self.linear1(scalar_mixing_weight[:, 0, :])
          linear2_output = self.linear2(self.dropout(linear1_output))
          sigmoid_output = self.sigmoid(linear2_output)
        elif self.i == 3:
          linear1_output = self.linear1(scalar_mixing_weight[:, 0, :])
          linear2_output = self.linear2(self.dropout(linear1_output))
          linear3_output = self.linear3(self.dropout(linear2_output))
          sigmoid_output = self.sigmoid(linear3_output)
        elif self.i == 4:
          linear1_output = self.linear1(scalar_mixing_weight[:, 0, :])
          linear2_output = self.linear2(self.dropout(linear1_output))
          linear3_output = self.linear3(self.dropout(linear2_output))
          linear4_output = self.linear4(self.dropout(linear3_output))
          sigmoid_output = self.sigmoid(linear4_output)

        return sigmoid_output


In [27]:
def get_scalar_mixing_model(num_bert_layers, i):
  gpu_available = torch.cuda.is_available()
  model = ScalarMixingWeightModel(num_bert_layers, i)

  if gpu_available:
    return model.to(torch.device("cuda"))

  return model

In [29]:
train_data

,Review,Sentiment
0,I like Mexican cuisine (not just TexMex but re...,positive
1,These Scripture Journals are an excellent reso...,positive
2,"The same ""ehh"" I felt reading Not Alone.<br />...",negative
3,The book is structured properly; It gives you ...,negative
4,This is a nice story of a Latino family’s jour...,positive
...,...,...
7995,It seems like people either blindly love or ha...,positive
7996,There seems to come a time when the liberal in...,negative
7997,wonderful story for Valentines day or any day....,positive
7998,"Disappointed, bought this last month, off to G...",negative


In [34]:
final_model = get_final_model()
train_model(final_model, 2, dataloader, train_data, learning_rate = 1e-4)

tensor(0.6413, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6862, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6140, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.3613, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.3469, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4116, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2444, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.1887, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4088, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2496, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.1940, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.3255, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2754, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 0 F1: 0.955625 LOSS: tensor(0.3859, device='cuda:0', grad_fn=<BinaryCr

In [48]:
qualitative_analysis_model(baseline_model, final_model, train_data, test_data)

Mismatched Reviews in Test Data:
Review: Wonderful book but not crazy about using the word hate with the kids...too strong a word for the fun topic. We just read it and choose a different word to use in its place | Correct Model: Final
Review: First let me say that I am new to the vegan diet, so I have collected a handful of books I thought would help me... having said that - this cookbook has NO photos.  :(  But... it still has a long list of useful recipes.  Most are listed gluten free and soy free - if that matters to you.  At the top of each page and under the recipe name it tells you this info...  I haven't found any strange ingredients (meaning you can't find what you need at the local grocery store - as we live in a small rural town) and no super special pieces of kitchen equipment... so I consider that a bonus.  I am not a trained culinary chef and I manage to understand the process of these recipes... I am a fan of one pot cooking (in all fairness some of the recipes use more 

(None, None)

In [49]:
# The raw result string
mismatched_reviews = """
Review: Wonderful book but not crazy about using the word hate with the kids...too strong a word for the fun topic. We just read it and choose a different word to use in its place | Correct Model: Final
Review: First let me say that I am new to the vegan diet, so I have collected a handful of books I thought would help me... having said that - this cookbook has NO photos.  :(  But... it still has a long list of useful recipes.  Most are listed gluten free and soy free - if that matters to you.  At the top of each page and under the recipe name it tells you this info...  I haven't found any strange ingredients (meaning you can't find what you need at the local grocery store - as we live in a small rural town) and no super special pieces of kitchen equipment... so I consider that a bonus.  I am not a trained culinary chef and I manage to understand the process of these recipes... I am a fan of one pot cooking (in all fairness some of the recipes use more than one pot - but served in one bowl..), so this book appealed to me.  I like the Red Bean Gumbo (however I replace the green bell pepper with a red bell pepper as I am not a fan of green bell pepper and I omit the okra - not a fan of that either)...  very satisfying. In the book it mentions that &#34;gumbo&#34; is the African word for okra!  Hmm... I like gumbo... but sans the okra. haha.  In any case, I purchased this cookbook and a handful of others to help inspire me to try to eat vegan.  Now... if we could get them to add photos and a spiral back (to help keep the book open while you are using it in the kitchen) it would be a 6 star.  :) | Correct Model: Final
Review: Sad. NOT funny. But DESERVES four stars.  I finished it BECAUSE it was well written, but I would not have read it had I known what was coming.<br />Some of the blurbs at the beginning of One's Company called it a Black Comedy.  I did not laugh once.  In fairness, the novel is well written, hangs together, has some great characters and Bonnie Lincoln and how she proceeds when she wins a bazillion dollar lottery is amazingly wrought.  But it is a sad, introspective book about a very damaged woman who needs to live as  a character in Three's Company to survive. Bonnie is a woman in her early twenties who has only ever had exactly one friend.  Both her parents are dead and you would not call her family one that was memorable in a good way.  She's recently gone through a horrible trauma, coming out of a situation as the only survivor-- if you want to call it survival.<br />Somehow, the old sitcom, Three's Company, comforts her and she becomes obsessed.  When she wins the biggest lottery ever, Bonnie creates an alternate world for herself of the Three's Company set and sets things up so that she will live alone there and be one of the characters from the show each year, moving from apartment to apartment, but always existing in the 1970s.. We already know the concept of this novel going in.  It's in the blurbs. Personally, the best thing about the book, is what I THOUGHT it was going to be. Something light and silly and a little pathetic but in a way that doesn't make you curl up in pain.  Well done. Wish I had not read it. | Correct Model: Final
Review: A chaotic tornado of characters. | Correct Model: Baseline
Review: I know Dan Jenkins is a coveted sports writer but man, can he ease up on the "n" word. I'm a golfer but I was turned off by it. Watch the HBO movie version. They substitute the "n" word with "ethnics" which is (sigh) obvious. In my opinion, Randy Quaid has the best golf swing for an actor. And there's a good nude scene by Corinne Bohrer. | Correct Model: Baseline
Review: 👍👍 | Correct Model: Final
Review: Purchased for my husband for Christmas.... he has always wanted to build a wooden boat.  He loves that this one has some actual plans that are good to consider as a beginner.  He notes that you'll probably need a couple of different books to compare and gather info, but this one is an excellent one to start with.  I would consider that a 5 star review from him. I always update if there is anything important to report.  I will say the book arrived with a folded corner and not &#34;packed&#34; in any way, along with other items I had ordered.  That part was a little disappointing. | Correct Model: Baseline
Review: I loved reading how mouse and Thomas came about. I'm not sure about Laura's father though. No one writes about Wizard fights like Jim Butcher. | Correct Model: Baseline
Review: Roberta did a good job reporting what the official line was. Alas, the truth lay elsewhere. | Correct Model: Baseline
Review: These items were picture books in my thoughts. Took my kid an hour if that to finish all three books.<br />Not worth the price. | Correct Model: Baseline
Review: My kid had a little trouble with the stickers | Correct Model: Final
Review: I like diff style idea to make but it is hard to understand for deaf people . | Correct Model: Final
Review: I choose this rating because I had to keep reading this book. I could not put it down as I wanted to know what happened next | Correct Model: Baseline
Review: This book is so cute and simple and the rhyming makes it even better. It was a board book as described and in perfect condition when it arrived | Correct Model: Baseline
Review: Common sense information. | Correct Model: Final
Review: Although I love LEGO and miniatures, I’ve never been particularly interested in building microscale LEGO models--there is too much loss of detail in the end.  This book, LEGO MICRO CITIES:  BUILD YOUR OWN MINI METROPOLIS!, illustrates the “problem”:  although the colorful endpapers show four different city models (Bridge Town, Hyperloop City, Steamworks, Upper Brick Side), the models and buildings all look very similar.  There are only so many different “looks” that you can achieve by combining the smaller LEGO elements.<br /><br />The book has complete instructions for eight different city models.  The ones not previously mentioned are Gilded City, Tomorrowland, Utopia, and Strange New Worlds.<br /><br />Basically, this book is helpful for LEGO fans who want to get started on building their own microscale architectural models (models along the lines of the official LEGO sets in the “Architecture” theme series).  It explains how to construct different types of tiny buildings (round, rectangular, or stacked skyscrapers, for instance), as well as how to create tiny but recognizable trains, ships, trees, statues, trees, etc.  There are also tips on how to solidly attach various components to the baseplate and other elements.<br /><br />Hardcore LEGO aficionados will enjoy looking through this well-done book.  However, it’s quite limited in scope, and of course you will end up buying bricks if you want to replicate the models shown in the book. | Correct Model: Final
Review: From the premise I was hoping for a [[ASIN:0380481650 Devil's Desire]] but by the end of the first chapter I was bummed out. I would say read the first chapter sample and if it works for you my guess is you will enjoy it because the rest of the book is more of the same. I felt the setting was spot on for time and place so I found that enjoyable. The main characters and their thoughts and dialogue just seemed jarring to me. I thought Grace, for being 26, and head of a household of 7 siblings thought and acted, at times, like a 16 year old. | Correct Model: Final
Review: Not at all what I had hoped for. There is little to no history or information about San Diego-1 page. It is certainly interactive and in a workbook format. | Correct Model: Baseline
Review: This book was disorienting, but not in an entirely unpleasant way. Wink, Poppy, and Midnight circle around each other in a strange story about hopes, fears, longing, and belonging. That, and tales - both told & read.<br /><br />Midnight is certainly the milquetoast, compared to Wink & Poppy. He only exerts his own will a few times in the book, and seems to doubt himself in all but the last instance (which is just a few pages from the end). Wink was mesmerizing, and strangely reminiscent of Luna Lovegood (from the Harry Potter books). Poppy was something out of a CW television show, which was probably the most distracting thing to me. Just too perfect and too cruel and too broken.<br /><br />Much of the story plays out like a CW show, too, in that there is just *so* much teenage melodrama and hormones. And it seems a little too self-aware, for lack of a better phrase. Like the book itself is preening for the reader.<br /><br />But it's mixed in with fairies and taro cards and hauntings, and ultimately a mystery about exactly who is who in this story. This is a quick, well-written book, easy to read in one sitting, and I definitely enjoyed it enough to care how it all ended. | Correct Model: Baseline
Review: I love the story. My children really liked the story. However, the way the story is written makes it an awkward read aloud book. The pictures in the book are pretty, almost like looking at paintings. | Correct Model: Final
Review: I love vegetables so I ordered this book because of the plant-based recipes and not because it is associated with an alkaline diet.<br /><br />I read this from cover to cover and selected all the recipes I would make. The first chapter is about the alkaline way of eating and recommends introducing such a diet to  reduce the burden on one's body related to  nutrient deficiencies, metabolic disorders, cardiovascular and digestive illnesses, etc.<br /><br />I decided to make the roasted potato hash. It smelled fabulous as I was cooking it. Now the recipes never take me the short time it says in the book. It took me a couple of hours to make this even though it said 15 minutes each for prep and cooking. Also, it asks for a large shallow pan. I grabbed a jumbo shallow pan and luckily I did because it was overflowing with the required ingredients. In addition, step four lists garlic twice but that is just an editing error. The resulting hash was excellent and very tasty. I am confident the other recipes will be too but I must ignore the prep and cook times because they never apply to me. I'm a very slow cook and can never cut up items like expected. For this particular recipe, it took me about one hour to prepare and 45 minutes to cook, not counting clean-up, serving, and other time.<br /><br />I really like this book and am glad I ordered it because there are more than a few recipes I find enticing. Note that there are hardly any photos in this book. | Correct Model: Final
Review: I haven't read this yet because I know from following his blog as his dear wife was struggling, I'll cry again - but I know they'll be reunited in Heaven some day, and that makes it easier to think about. | Correct Model: Final
Review: This is an unbelievable story about oppression of a woman and her children in an age where enlightenment of the human condition is unknown to the FDLS world. It is a story well told by Carolyn Jessop, whose bravery in the face of insurmountable odds that would overwhelm even the most courageous souls among us and, of her success in escaping with her eight children, and emerging victorious from a life of submissivenss to a cult's polygamous lifestyle that one finds hard to believe exists in the 21st century. | Correct Model: Final
Review: I wish I had know about the Sar-El Volunteer Program much earlier in my life. I would have loved to have been a part of it, but now my health prevents that. | Correct Model: Final
Review: Minimal use marks and still in great usable condition. | Correct Model: Final
Review: I read this after it was recommended by a friend. I thought it was fast paced & interesting. I'm not a big fan of &#34;amnesia&#34; murder stories because it's all too convenient. It's also frustrating! The chapters are written in different point of views. Not only do you have to pay attention to which character is narrating the chapter, but what the date is for that chapter, too. I also found it sad that I didn't like any of the characters. Every time I thought that maybe a character was okay, something would occur that would change my mind. All of the main characters had some kind of mental health issue. It did manage to hold my interest and I didn't want to stop reading. It was suspenseful and entertaining. | Correct Model: Baseline
Review: This is another cute American Girl kit. The kit comes with 2 doll sized t-shirts. There are some stickers and stencils for decorating the shirts. The book shows how to decorate the shirts. In addition to using the stickers, it shows how to make felt decorations on the shirts and how to paint on them. The stickers aren't really reusable since they loose their stickiness. If you paint on the shirt or glue the felt to them, then the shirts can't be decorated again.  Younger readers may need some adult help, as mentioned on the back of the kit. | Correct Model: Baseline
Review: Not excited about the content, but I work on reading programs and have very high standards. | Correct Model: Final
Review: Chernobyl is a huge tragedy that I remember reading about and crying about when it happened. Strangely enough I felt detached from the book and the miniseries.  Not sure why I couldn't relate. I haven't found he right book to relate this tragedy too. I remember photos in the newspapers at the time and powerful reporting. | Correct Model: Final
Review: I thought it was just a coloring book and it's more of a story book with a few things to color. | Correct Model: Baseline
Review: In the last two years I’ve become interested in the years between the World Wars - fiction and nonfiction, especially the Weimar Republic era. The setting, a rural area and not being in Berlin, made me eager to read this book.<br /><br />First the good: I really liked Fallada’s writing style - rapid-fire storytelling, often driven by quick and energetic dialog that forced me to concentrate to “keep up.” I liked that. The descriptions of the town, its people and their relationships and conflicts were also well-written and engaging. I have a few other Fallada novels on my list, and I will give them a read.<br /><br />The not-so-good: The first 50% of the book was tight and exciting, with a good building of the coming conflict within the town. But the almost overwhelming number of main and secondary characters (the Dramatis Personae did help!), and the many sub-plots and little intrigues got to be a bit much during the book’s second half. And the conclusion - after so much conniving, chicanery and double-crossing  within the community - was a bit of a fizzle, flat and disappointing. At almost 600 pages, it seemed bloated, and maybe 100-150 pages too long. | Correct Model: Final
Review: This book isn’t extremely well written. It is an interesting non-fiction chapter book. Some kids read more readily when they are reading non-fiction. it Is about a third grade level, maybe a little under third grade. | Correct Model: Baseline
Review: This is not your typical romance at all. The main character is a recent widow. She drinks a lot so she doesn't have to feel anything. There were times when I didn't really like her, but other times I wanted her to find happiness. The hero of the story has his own problems and issues. He's a Hollywood actor and just as screwed up as she is.  In addition to excessive drinking, there's a lot of swearing - especially the &#34;f&#34; word. There's some sexual content, but it's not too graphic. Overall,  I thought it was well written and engaging. I started reading a couple of chapters a day, but then on the third day, I couldn't put it down. | Correct Model: Baseline
Review: I have been a fan of Jane Goodall for a long time.  This book demonstrates how involved she is with conservation and her quest for knowledge regarding the various species of animals, plants, fish, and insects on our planet.<br /><br />This text highlights just some of the animals that are or were:<br />- extinct in nature<br />- almost extinct in nature<br />- those making a comeback in nature<br />- those that were thought extinct (the Lazarus Syndrome) but have since been rediscovered<br />- etc., etc.<br /><br />Some of the animals (and other rarities) you will become acquainted with are the Pere David's deer, Sumatran Rhino, some Asian Vultures, Panamanian Golden Frog, Crimson Spider Orchid (flower--picture only, no story), Coelacanth (fish), Lord Howe's Island phasmid (stick insect), the Tahina Palm (plant), etc., etc.<br /><br />The latest conservation efforts are presented in short book report form for each of the many extinct or almost extinct animals, plants, fish, and insects. The only reason I rated the book four stars is because you feel you are reading individual book reports instead of an engaging flowing narrative.  However, I am not sure this book could have been written any other way.<br /><br />A website was mentioned as an extension to this book but no website address was given.  As of today I could not find such a site during a Google search.<br /><br />"Hope for Animals and their World..." is definitely an enducational read for someone interested in the natural world. | Correct Model: Final
Review: Eve just gets better and better. I love that she’s not perfect, has her own concerns and makes mistakes. Looking forward to reading about her next cases and how she handles them. | Correct Model: Baseline
Review: I tried the audio version of this book and found it too confusing.  After several months I decided to try it again,  this time in written form and I loved it.  The structure is almost like a play, which in the end works for the emotional impact of its insights.  I am so happy that I read it. | Correct Model: Baseline
Review: Interesting, but not quite as much as I'd hoped for.... | Correct Model: Final
Review: Well,this book is even better than the first. Jinx is learning so much more about her powers. I can't say too much with out giving away the surprise,but there is so much more information about the other characters in Jinxs life that comes out ...but she doesn't know yet! However,her and her bff Tori,find out why they are so close..it's not just a coincidence . They way the author lets us know these surprises is written so well..it definitely sets the stage for the next book and I can't wait to read it.!!! I voluntarily leave reviews for all the books I read whether I pay for them or get them free ... | Correct Model: Baseline
Review: Very fast book.  Didn’t take my child long to finish. | Correct Model: Final
Review: FALL OF ANGELS (AN INSPECTOR REDFYRE MYSTERY) introduces Barbara Cleverly’s new detective, John Redfyre.  He’s sort of a cross between Inspector Lynley and Inspector Morse, because he’s a well-connected, educated policeman assigned to a university town (Cambridge).  Redfyre is unique, though, because his series is set in Cambridge of the 1920s.  Be warned that this is a “feminist” novel—the plot centers on a group of strong women who are working to achieve true equality after the 1918 grant of limited women’s suffrage in England.<br /><br />In the novel, a female trumpeter (a highly controversial choice of musical instrument!) is nearly killed by a suspicious fall down the steep steps leading from the choir loft at the St. Barnabas College Chapel.  Redfyre is in attendance at the trumpeter’s ill-fated performance, and cordons off the area near the steps, treating it as a crime scene.  Soon enough, he discovers a second attempted murder of the trumpeter; and actual murders of various women who are loosely connected with the trumpeter follow shortly.<br /><br />The novel is well-written, with fully developed characters and solid historical background.  I like Inspector Redfyre every bit as much as Joe Sandilands.  The plot of this novel leaves something to be desired, though.  It’s highly complex, with lots of unexpected twists and turns, and it has something of a surprise ending.  However, the story loses its momentum and begins to peter out in the last 75 or so pages, so that the ending is unsatisfying.  This is a good mystery/thriller, but it’s definitely not Cleverly’s best. | Correct Model: Final
Review: This is not my usual genre, but I was willing to give it a try. The first 100 pages moved very slowly. There were times that I thought I was going to quit, but I trudged on. Things did pick up and I found it difficult to put down during the last 100 pages. Since I haven't read a lot of books set in the late 1890's, I didn't have much in the way of expectations. It did give me sort of a Sherlock Holmes feel to it. The language was a little frustrating because it was very wordy and dragged out which I attribute to the niceties and formalities of the time period. This did cause me to skim at times, but then I would miss something and have to go back and reread some. The subject was a bit supernatural, but interesting. Overall, I did enjoy it, though I'm not eager to dip into the genre again anytime soon. | Correct Model: Baseline
Review: Love these pop- up books, but I do have to supervise the little ones with them. | Correct Model: Final
Review: Weyu | Correct Model: Baseline
Review: this is so very similar to one of his other novels, a different ending though. I enjoyed it but had I known how closely it would be to the other novel I probably wouldn't have bought it. | Correct Model: Baseline
Review: All told by Jon. After 20 years of marriage Jon (husband) & Timmy (wife) divorce.  He is a writer, she is a doctor. They had married after cheating on their partners. So the reader can see the possibility exists. Jon tries to figure out what happened. Hello? Timmy wanted change, end of story. This thing read like Jon was told by his therapist to write a dairy to get over his wife leaving him. TMI.  The end of the marriage began with John becoming a shut in and Timmy needing a social life. John's career tanked at the newspaper he worked at and he began working from home, Timmy needed space. Jon became needy for years on end. It didn't help that from the beginning Jon began openly fantasizing about strangers making love to his wife. | Correct Model: Baseline
Review: This is a cute "moments of love" desk calendar for 2020.<br /><br />- It stands on its own by folding the front cover halfway around back and then connecting it with the invisible magnetic strip in the center of the back cover (photo 1).<br /><br />- Each page is perforated and can be cleanly ripped off when the day is over.<br /><br />- The comic takes up the entire page and the date is only on the bottom. There really is no room for notes.<br /><br />- Each day has its own page except for Saturdays and Sundays, which are combined on one page. Pictures 3 and 4 show a sampling of the comics on February 15/16 and May 18. | Correct Model: Baseline
Review: I didn't want to put this book down! It grabbed my attention from the get-go. I finished it in a day. I would classify this as a chick lit book rather than a romance. The story starts when Dannie, is asked a simple question at a job interview - where does she want to be in 5 years. Shortly after that, something happens that causes her to doubt the path she's taking. Without giving anything away, she resists deviating away from her plans. I found it to be quite enjoyable! | Correct Model: Baseline
Review: Just ok pictures were like for kids. | Correct Model: Final
Review: Need to pay attention to what is happening in the world today! | Correct Model: Baseline
Review: If you are geniunely looking for useful information on growing for medical use, this book is a must have. I knew nothing about how to grow what I needed for medical use.  This book answers all my questions and more. | Correct Model: Final
Review: Not bad but they’re a bit weird | Correct Model: Baseline
Review: Tactile and colorful | Correct Model: Baseline
Review: This might have been much too low level for our family.  We love to collect Spanish language books and we have found that it is the absolute best way to increase vocabulary, but this book might have been far to young for us.  It is very, very basic with colors and shapes and perhaps more suitable as a toddler coloring book.  I'm sure it is perfect in the right hands but unfortunately not the right fit for us. | Correct Model: Final
Review: Some assembly required!  KRUSTEAZ CINNAMON CRUMB CAKE & MUFFIN MIX tastes good, but requires more preparation steps than you expect from a cake or muffin mix.  If you're making crumb cake, you have to layer the cake mix and crumb topping, swirl the batter with a knife, and then sprinkle on a final layer of crumb topping.  If you're making muffins, you have to fold half the crumb topping into the cake mix, and then sprinkle the remaining crumb topping on top.<br /><br />My husband loved these.  I loved the cake part, but felt that the cinnamon/sugar/crumb topping was too sweet.  However, you can't really go wrong with Krusteaz mixes, particularly [[ASIN:B0016BU7AK Krusteaz Banana Nut Muffin Mix)]] and [[ASIN:B0016BU7GO Krusteaz Cranberry Orange Supreme Muffin Mix]].<br /><br />For a crumb cake or 12 standard muffins, you need one egg, 2/3 cup water, and one package of the mix.  My muffins took a couple more minutes than the package directions called for, when I used the convection oven setting on an electric range.  I purchased this particular Krusteaz cake/muffin mix at my local grocery store. | Correct Model: Final
Review: Although I liked the ending of this book with the Vulturi, I really struggled with the beginning because she was such a weak-minded girl.  I was glad to see her snap out of it in the end.  I am hopeful the next book won't be as bad.  I liked the first book in this series a bit better than this one. | Correct Model: Baseline
Review: DON’T BELIEVE IT is a complicated mystery/thriller with a lot of “chapters” printed entirely in italics--which gives you some idea of what you as the reader are in for.  The basic storyline is that a woman (Grace), who was convicted of murdering her lover on Gros Piton in St. Lucia, Eastern Caribbean, contacts an old friend (Sidney) and asks her to create a TV documentary about the case.  Grace hopes that Sidney will uncover new evidence that will exonerate her and cause her release from prison.<br /><br />Sidney obliges, and the italicized chapters are summaries of the resulting individual TV episodes as they air.  Thus the novel is multi-layered, with retellings of the basic murder events that are nestled inside new accounts of ongoing events (including the public’s reaction to the TV episodes).<br /><br />The plot is twisty, with some unpleasant things in store for the helpful Sydney.  There are many abrupt shifts among characters (e.g., Grace, Sydney, Gus, etc.) and locales (St. Lucia, New York City, rehab hospitals, etc.).<br /><br />In the end, the revelation of the killer’s identity and true motivation, and of why/how the novel’s various murders were/are covered up, is unbelievable.  (“Family first” and “blood is thicker than water” are strong motivations, but a good murder mystery needs more.)  The novel ends with a cliffhanger of sorts.<br /><br /> As a regular reader of mass market mysteries, I find a novel that is constructed like this one to be too much work to be enjoyable.  It's not particularly entertaining, and it’s slow going. | Correct Model: Baseline
Review: My favorite science in school was "earth science" and I was excited upon seeing this book as it looked as if it were an interesting geological history of the planet. I never like to admit that I didn't finish a book, especially one that I had sent for that looked so promising, and don't like to submit a review of a book I didn't finish, but I must be honest...I never have finished this book. Maybe it improved in the last half. I still have it and perhaps my tastes will have changed in the future and I will tackle it again. | Correct Model: Final
Review: This book is aimed at a more traditional fantasy league.  The league I am with allows a person to select a team from players throughout baseball history and does not involve a draft now the selection of a local commissioner.  Although I did find parts of the text applicable, it just was not aimed at my league. | Correct Model: Baseline
Review: I'm not entirely sure I would label these for middle school. They seem upper elementary to me, at best. Some of the titles even perked the interest of my kindergartener. He, of course, cannot read them himself. There are a range of topics and these are really great for kids who dislike fictional reading. Each page is full color and glossy with lots of pictures throughout each book. I think these are a decent resource, but the set seems a bit more random than I would necessarily like. (I would rather buy a sociology set or a set on the American Revolution than a complete hodpodge of random topics.) | Correct Model: Baseline
Review: When I first received this book to review, I saw that the author had written a previous account of her early childhood in foster care, "Somebody's Someone."  I ordered this book from the library, thinking it would give me insight into the current book, and it does.  If you have time, I urge you to read "Somebody's Someone" first, as it will give you background and context.  Frankly, it is better written as well.<br /><br />However, I do not want to take anything away from "Someone Has Led This Child to Believe."  Regina continues the account of her life through adolescence and early adulthood, and it is difficult for this middle class white woman to comprehend how she survived with her spirit and mind intact.  I am sure writing these books has been cathartic for her, and I applaud all those who tried to help, and especially those who succeeded.  Most readers will be appalled at the abuse, physical, mental, sexual, psychological, that Regina lived through.  Essentially abandoned by her mother and father, sent from foster home to group home to the homes of parents she did not know. Expected to clean the house and care for younger children when only a child herself--how she survived this is nothing short of a miracle.  And the miracle comes in the form of people who saw something of value in Regina and led her to believe in herself and her future.<br /><br />This book will appeal to those who are interested in children, such as foster parents and teachers, but it is not an easy read.  It is a good and necessary book, however.  Highly recommended. | Correct Model: Baseline
Review: I could go back and read this book some more, has some good tips for those esp. knew to the DSLR world and want to slowly get wet without being bogged down with too much new terminology. | Correct Model: Baseline
Review: I have to admit I was rather surprised at what this book ended up being.  It is a very deep existential look into dogs and where they fit into our culture.  Along the way there are a lot of interesting facts such as Trump is the only president ever to not be a dog owner, which may make him uniquely the way he is. Also, you learn how monotheism was the downfall of our esteem for dogs as godlike creatures which are now loathsomely low on the single god totem pole.  They went from revered to downtrodden.  It's a dense, almost academic book, which if you like that kind of thing, is filled with little delicious tidbits.  But if you are not into that thing I would think it would be heavy, hard reading, especially for the less academic dog lover.  It was an interesting read for sure, but I don't think it's for the mainstream masses. | Correct Model: Final
Review: This is a coloring book with stickers | Correct Model: Final
Review: My 5 1/2 yr old Grandson loves to do Mazes, and found this fun, but pretty simple for him.<br />He finished this book in maybe 30 mins, and was joking around doing it.<br /><br />There are allot of stamps in here, that I think are more for girls!<br />In the back of the book it shows all the pages In the book and what page there on.<br /><br />This book is well done, but I think it’s more for 3 and 4 yr old!<br /><br />4 stars | Correct Model: Baseline
Review: I purchased this for my husband as he has been talking about possibly making one of these so I thought he'd like to look at this book.  If for nothing else maybe he can learn about the sport of landsailing.  The book mainly has black and white photos... not a super high quality print.. but it will serve the purpose. | Correct Model: Baseline
Review: What a concept - I could not put this book down!  Would recommend it 100%, and plan to re-read in months to come.  Never thought I'd be pulling for the Devil - LOL | Correct Model: Baseline
Review: I couldn't wait for this book to come out.  I read all of Stephen King's books, and this one did not disappoint.  I love the fact that I can pre-order. | Correct Model: Baseline
Review: I received this book as a gift from my dear daughter and I appreciated it and enjoyed it -- up until page 119.  There, authors Dutcher's release their version of CWT  -- Critical Wolf Theory.  No, they don't use that term but their anti Christian message is crystal clear.  They write, "With the rise of Christianity... it reached a population [Europe] that increasingly envisioned itself as separate from nature.  It couldn't have helped that Christians adopted the motif of God the Shepherd."  No, Mr. and Mrs. Dutcher, Christians didn't "adopt" the motif of God the Shepherd.  The 23rd Psalm, one of the most beloved and moving scriptures, was GIVEN to us by God.  Christ himself said, "I am the good shepherd." (John 10:11)  Shepherding was a common occupation during Biblical times and it was something everyone could understand; and yes, wolves did attack flocks of sheep; and yes, the job of the shepherd was to protect their flock.  It is a symbolism that I, and billions of people, cherish.  Never in my life have I imaged this symbolism as a basis for hatred of wolves.  In fact, I support wolf reintroduction, I have "adopted" a wolf at NWF, and I give regularly to NWF.  Your indiscriminate bashing of Christians is unwarranted, unwelcome, and diminishes your otherwise praiseworthy message. | Correct Model: Baseline
Review: There is little of Inspector Sejer in the first half of the book and it is far from the police procedural that draws this reader to Fossum's work. I could not tolerate the main character's endless whining and self-pity. I couldn't bring myself to care about him and only wanted him locked up and shut up.<br /><br />Such misery! A gloomy story told from the perpetrator's POV. His state of mind, in all it's denial and self-centeredness is explored thoroughly. He disappoints his 16 year old daughter and breaks her heart over and over.<br /><br />Try one of the other titles in the series and you will be hooked:<br /><br />Inspector Sejer / Inspector Jakob Skarre series:<br />1995 - In the Darkness; English translation 2012<br />1996 - Don't Look Back; English translation 2002<br />1997 - He Who Fears the Wolf; English translation 2003<br />1998 - When the Devil Holds the Candle; English translation 2004<br />2000 - Calling Out For You; American translation: The Indian Bride, 2007<br />2002 - Black Seconds; English translation 2007<br />2004 - The Murder of Harriet Krohn; English translation 2014<br />2007 - The Water's Edge; English translation 2009<br />2008 - Bad Intentions; English translation 2010<br />2009 - The Caller; English translation 2011<br />2013 - Carmen Zita og døden<br />2014 - Helvetesilden | Correct Model: Baseline
Review: This book was part of our survival unit in seventh grade. I was bored and anoyed the entire time. There was a lot of sitting in the crashed car and siting fallowed by more sitting. I hope today they give the kids something good like when all hell breaks loose by Cody lundine or something that may actually help in a survival situation. | Correct Model: Baseline
Review: I am always open to reading a translation of such a book where it is a best seller with an established author.  With such an opportunity...What did I think?<br />First, I had to adjust to the cultural theme difference and allow myself to open up to the prose and angle of the author. For me it took me 1/4 way into the book to understand the jump from story telling to 'interviewing technique' from another character's angle...<br />Second, the key character was centered around Jenzhen, who had this amazing brain that could adapt to anything mathematics!<br />Third, it was a human assessment of how society could exploit such delicate souls with these extraordinary gifts, and then can abandon them ....it was a sad ending, and one that was depicted with much truth of how society works.<br /><br />It is not much of a thriller as we know it in the west.  But I was happy to have a chance to read this book that provided an insight of the Mao's revolution at the time too... | Correct Model: Final
Review: I had difficulties deciding between 3 and 4 stars. In the end, I went with 4 stars because I did like it. I have been a fan of the Pendergast novels from the beginning. They are not my usual genre, but I find them addicting! This one is definitely one of the weaker ones. First of all, Pendergast doesn't even make an appearance for at least the first third or more of the book. His chauffeur/right hand man, Proctor is the lead character during that time. I enjoyed reading more about him, but then he abruptly disappears in the story line for a good chunk of the book. The book is fast paced and exciting, but it felt like it was the same old story. I'm not giving up on the series yet, but I am thinking it might be time to retire the beloved Agent Pendergast. | Correct Model: Final
Review: Although the book is very well written and at time entertaining, it did not provide me with what I wanted. The useful information to set up a baseball betting system is scant and ceases with the sixth chapter.  However, I am reading those first chapters for the third time and find them insightful and intriguing.  The book just left me wanting. | Correct Model: Final
Review: Supposed to be a used book but you couldn't tell by looking at it. Arrived within the scheduled delivery time. My son is happy with it. I try and purchase used when the option presents itself. Less waste when we reuse items instead of always buying new. Thank you for making that an option. | Correct Model: Baseline
Review: It's like reading a game of boggle or un-scrabbled! | Correct Model: Final
Review: collection of quotes - read it on the plane on the way to Heathrow. Got me into the mood for our English vacation adventure. Cute little book but just okay. Nothing really special. | Correct Model: Final
Review: "Unseen" by Karin Slaughter is only the second novel I have read by this author.  The first (sorry, I cannot recall the title) was great--fast-paced, intriguing, and not a Will Trent book.  Like that book,  however, "Unseen" takes place in Macon, Georgia, and has many good, bad and truly evil cops in it.  It was a slow read at first, because most of the characters are cops, and there is just not enough variety, at least for me, to make it interesting.<br /><br />The exception to this is Dr. Sara Linton, who is in love with Will Trent, an undercover agent in this book who is likewise in love with her.  Unfortunately, he is still married to Angie, who is said to be mentally ill, so he is not free to pursue a relationship.  Another character, Lena, is a police officer who is hated by the reader but loved by her fellow police officers.  This disparity is incomprehensible.  Lena was Sara's deceased husband's partner on the police force, and Sara blames Lena for her husband's death.  Although she is not at all likable, she adds a great deal to the novel.<br /><br />In this novel, Sara thinks Lena is trying to kill her own husband, Jared, who is Sara's stepson.  This four-star novel has many twists and turns as the other police officers think Lena can do no wrong.  A reviewer in our local newspaper  recommended this book, but commented that it is very helpful to  have read the previous Will Trent books.  I completely agree with this.  You will be much happier with this book if you have read the other books in the series. | Correct Model: Final
Review: This isn’t the best book on child development, but it has proved to be a decent resource for finding information on specific topics. I appreciate the content, though more as a reference than anything else. | Correct Model: Baseline
Review: I love psychological thrillers and this was a really good one. I just hate the way it ended. I hope there's a follow up to answer some questions. | Correct Model: Baseline
Review: GROWN-UP ANGER:  THE CONNECTED MYSTERIES OF BOB DYLAN, WOODY GUTHRIE, AND THE CALUMET MASSACRE OF 1913 purports to be a “cultural study”, but it’s so personalized--starting with an opening (18-page) chapter about the author’s gut response (at age 13) to Dylan’s “Like a Rolling Stone”--that it reads like a memoir.  Starting from that remembered response, author Wolff embarks on an epic quest to discover the source of Dylan’s anger, and recounts all the details of his quest in this book.<br /><br />Wolff settles on the anger source as the “1913 Massacre” that Woody Guthrie sang about (because Dylan used Woody’s same melody in “Song to Woody”).  The massacre was a real event, in which 73 children of striking copper miners and 1 adult suffocated in a Calumet, Michigan meeting hall stairwell.  Someone opened the hall doors and yelled ”Fire!” (a false alarm), and then anti-strike men held the exit doors shut from the outside so that none could escape.<br /><br />However, to take this single event, and Guthrie’s song, as the source of the anger expressed by folk singers in the mid1960s (as “have-nots”, against the “haves”) is a stretch, to say the least.  If anything, author Wolff's “revisionist history” shows that you can always find apparent “connections” between things, if you try hard enough.  Wolff tells us (in chapter 1) that his four years spent at Harvard College were wasted on him, and I would agree. | Correct Model: Baseline
Review: Start by saying I love Lucas Davenport books. BUT this is not even close to the best of them. I miss all the people we are used to, Del, Shrake, Jenkins, Virgil, Sandy, Rose Marie, & even Weather (who I could do without) & the governor. The Bad Guys are given way too much time - they aren't interesting at all, unlike most of Mr. Sandford's criminals. The plot is very sketchy & pretty uninteresting. Letty is a little flaky as usual & that part was ok. It is worth reading but don't look for the usual "can't put down until the last dog is dead". It is easy to put down, but also easy to pick up again. So, I am again waiting for the next Prey book & am pretty sure it will be back to the best ever mystery series. We are all very interested in what Lucas is going to do now that he is out of the BCA, & hoping that it doesn't mean that the series is ending!! | Correct Model: Baseline
Review: Probably one of my favorite Uncle John's bathroom readers. Lots of interesting facts I never knew about PA and I've lived here all my life. Flew through this book....didn't want to put it down. | Correct Model: Baseline
Review: I rolled the extra bucks for the spiral binding - which is super useful in keeping the book open as you are using it. I personally prefer the spiral binding whenever I purchase a cookbook. I collect cookbooks - even though I get lots of recipes online - there is nothing like holding and browsing through a cookbook for inspiration (my opinion) This cookbook is ground breaking -  I didn't see any recipes that knocked my socks off, but I am new to "pressure cooking" so I wanted a real basic start and this cookbook is definitely that. | Correct Model: Baseline
Review: Not for the beginner at all..unless you really want to slave over it.  But for the intermediary and up, these are a great collection of the 80's pop scence!  Great for guitar and piano, and the lyrics are included.  I wish it was on a comb binder, but for $8 you can get it bound on a comb from KINKO. | Correct Model: Baseline
Review: What I mean is, one of the main characters has a talent that the main character in these types of stories (don't want to spoil it) and the focus isn't really on him (I can't remember if it's in the description or not), at least not in this first book.<br /><br />The characters were well-fleshed (I wanted to smack a couple of them, which is a good thing. It means I got caught up in the story) and the plot was well-written. | Correct Model: Baseline
Review: This was my first Michael Palmer read, and from the accolades, I was expecting something exceptional.  Resistant had an exceptionally imaginative premise, a good story that moved right along, characters that were well-fleshed out, settings that transported, but, instead of enjoying the novel, I was distracted by the political aspects.  It was difficult to read without feeling disappointed that the author of a really good book had to make his political leanings so apparent.  I've read many a non-fiction book dealing with present and past politics - and I enjoy them for what they are.  I expect to deal with the political bent of the writer.  When it comes at me packaged as a novel, I don't anticipate feeling defensive while I'm reading!<br />So, I didn't finish the book, which is a rare thing for me.  If you don't share my delicate sensibilities, you will probably enjoy this book.  It is well-written, and I can understand why it was well-received by the publishing community. | Correct Model: Final
Review: "My journey with Farrah" is a book about how Farrah's best friend Alana helped Farrah during her battle with cancer--from the time she was diagnosed until her death.  It is written in diary form as if you were reading Alana's diary.  From Alana's point of view, you get an inside peek at Farrah's cancer battle.  The book demonstrates to me the love Alana and Farrah had for each other.  The story will humble you as you gain insight into how an extraordinarily beautiful and down-to-earth woman could be ravaged by such a disease.<br /><br />I've read most of the negative reviews about this book, but did not have the same reaction as the other readers had.  Although at times Alana can come across as selfish, I don't think this book would have been "true" had she not revealed her innermost thoughts that make Alana a human being like anyone else.  I commend Alana for voicing those thoughts even though most might recoil at them.  Since Ryan (Farrah's significant other for 30+ years) supported Alana in the writing of this book, I believe Farrah would approve.<br /><br />If you are looking for a book about Farrah's life, this is not it.  However, if you want to know all about Farrah's battle with cancer these last few years through Alana's eyes, then this book will give you a glimpse of Farrah's struggle along with what Alana's life was like during that time. | Correct Model: Baseline
Review: This is a timely story with its app development theme. This was an easy read, but it was a little boring at times. The characters were alright, but I didn't love any of them. I think the premise was good and it had a lot of potential, but in the end it fell short. I did like it (not love it). | Correct Model: Baseline
Review: BEAUTIFUL ANIMALS: A NOVEL is a literary mystery/thriller, which is a genre that I generally don't care for.  However, the novel does a superb job of capturing the suffocating, claustrophobic, heat-infused atmosphere of a small Greek island (Hydra) during the summertime.  In a way, the island itself can be blamed for the ill-conceived burglary scheme that goes horribly wrong, and that produces disastrous consequences for the main characters (Naomi, Samantha, Faoud).  It is the stultifying island that nurtures the seed of rebellious Naomi’s dislike of her family's lifestyle, and that allows the seed to grow, flourish, and bloom.<br /><br />The story is told almost entirely from the viewpoints of Naomi, Samantha, and Faoud.  The three are thrown together by Fate--and confined on a self-contained island that is populated by tourists, wealthy expatriates, and year-round native residents.  The events would never have occurred but for the two young women’s chance meeting, their friendship born at least in part from shared boredom, and the continuing presence of the blazing, blinding Greek sun that can affect people's thought processes.<br /><br />The story is slow-moving, especially in the first third of the book that mainly paints a beautiful picture of Hydra during the summer tourist months.  However, the plot picks up speed once the burglary scheme is hatched.  Ultimately the novel succeeds at drawing the reader into the events, and in making the burglary plot seem almost reasonable--perhaps even inevitable--given the island’s limited options for entertainment and its smothering atmosphere. | Correct Model: Baseline
Review: This is great, but doesn't have as many items as you would expect.  It doesn't contain items that you would expect to find in your grandmother's attic, unless she is wealthy and randomly stores expensive items.  It was fun to look at and if I happen to run into some of these items, mostly made in England, I hope I remember them! | Correct Model: Baseline
Review: Going to Rome later this month and needed a guide book for my visit. I had signed up for several tours which were meeting at obscure tourist points. I had googled the internet to try to find where these places were and was only able to find one of them. I had figured that I would need to get to Rome and ask someone in the hotel (in my very broken Italian) where it was that I needed to go. In comes the Lonely Planet guide to Rome. First thing I did was go to find the locations I couldn't find anywhere else. As it turns out, the meet-up places I had been looking for were actual 'local aliases' that if you were a Rome resident you would know. The Lonely Planet guide had these listed both ways and I was not only able to read about the locales, but also locate them on the enclosed street maps. Kudos to Lonely Planet and their terrific tourist guides. I had used them before in Paris and Ireland, but never had needed them like I did this time. And forgetting all this personal information I needed, rest assured there is plenty of other tourist info like entry times and fees for tourist sites and reviews on restaurants as well. If you are going to Rome,  you need to bring this with  you! | Correct Model: Final
Review: I love this book.  I was not blessed with artistic talent. Music yes. Drawing no. Lol. I have several of her books now & I fully intend to buy more for myself, my 5 kids, & my grandkids. We all love arts & crafts & these books are just awesome. My only issue is I haven’t brought myself to work in them yet! Lol. I use a separate sketch book!  I’m old school. Taught never to write in books or deface them. The only time I do is if the grammar or punctuation is wrong or the idiot author has now said the killer has blue eyes where previously they said green. Plot errors drive me crazy. Those times I will fix the mistakes. Lol.  So I’m hoping to bring myself to actually work & draw in the books. Lol.  I’m hoping she makes a lot more of them. | Correct Model: Final
Review: Purchased for an older adult that had zero smart phone experience. Starts with the most basic of basics and is written in an easy to follow way for beginners. Great guide for the smart phone novice! | Correct Model: Baseline
Review: The readability level was higher than expected. | Correct Model: Baseline
Review: This item was money well spent.  it is useful for groups, individuals, reteaching... yes, teachers cand make their own words problems but this is handy. | Correct Model: Baseline
Review: Was a good read, although, not as good as the other two.  Ended without disappointment. | Correct Model: Baseline
Review: This book starts slowly, but gains speed to the point where it's hard to put down. My daughter and I both read this, so this includes thoughts from both of us. One thing that I found interesting, is that different things were revealed as the book moved along and it answered questions that I didn't know that I had. The love story wasn't satisfying in the sense that I didn't care for the boyfriend and preferred another guy in the story over him. As a parent, it was a little off putting that the characters were smokers. There's also some sexual content. The sex wasn't just a reference or a quick glossing over, but longer than I expected. It's not explicit though. My daughter says that she wouldn't recommend it to anyone under high school to read it. The details about free diving were interesting and my daughter want's to try it now. We both liked it and found it entertaining. | Correct Model: Baseline
Review: The composition was Splendid.  The plot was unfinished. I don't require a happily ever after.  But some sort of conclusion was needed. | Correct Model: Baseline
Review: Large holes in the plot.  An easy read if your objective is just to pass time and not think too hard. | Correct Model: Final
Review: I live in Georgia and found this guide book to be okay, but not comprehensive. For it's size, there's no way it could include everything, especially since it covers 3 states. The book is a good size, but the paper is thin and not very sturdy. Guidebooks used to have heavier, glossy paper, but this does not. The information it provides is good, but there's so much more that's not included in it. For example, Savannah is one of my favorite places to go and two of my favorite restaurants, both of which have been there for years, aren't mentioned. I do like the idea of having just one book if I was traveling the three state area, but if you are just going to one state, I would suggest a more comprehensive guide book. | Correct Model: Final
Review: THE RISE OF ATHENS:  THE STORY OF THE WORLD'S GREATEST CIVILIZATION is &#34;narrative history&#34; that attempts to relate the story of Athens as it was experienced by the ancient Athenians who lived through the actual events described in the book. Because Athenians believed that the stories told about Theseus and about the events of the Odyssey and Iliad were actually true, author Anthony Everitt's narrative incorporates these fictional accounts into this &#34;history&#34;, right along with the more factual accounts of important events in Athens' history that were described by ancient historians.<br /><br />The book is extremely readable, but it's &#34;dumbed down&#34; history that cannot be relied on by any serious student of the history of Athens.  Fact and fiction get hopelessly mixed up in a presentation like this.  I like this book for its entertainment value, as &#34;a rollicking good yarn&#34;, but not at all as a history book.  The ancient Athenians were fascinating people, so the book is worth reading for pleasure, or as a young person's introduction to Ancient Greece, but I can't see it as a serious work of history. | Correct Model: Final
Review: These card readers are really nice.  They are laminated and you get 6 of each laminated card with 6 readers. The card folds in half to make it book-like.<br /><br />The readers are on Chimpanzees, Ants, the Inuit people, sounds, the African elephant and weird jobs.<br /><br />This would be a great addition for a teacher who happens to be teaching on Chimpanzees and the other items, afore mentioned.<br /><br />It would be nice to include a quiz at the end of each book. | Correct Model: Baseline
Review: Their Super Bowl wins against the New England were pure luck no skill. | Correct Model: Baseline
Review: I tried this disc in 4 PC CD players, 1 progressive scan JVC DVD/CD player and a PS2. On the PC's you saw a graphics page with areas in yellow. Lower right section had two areas to click. One for a video of Ms. Gertrud Huber playing the Harry Lime Theme by Anton Karas. The other had web site links. The tracks would not play. The (20 tracks) would be visible as blocks in the PS2 screen, but each had to be selected and there was no play through. There was the addition of dialogue at the beginning of each track (following the film/music structure). As the CD would come up (on the PC) you would hear the Harry Lime Theme playing. Amazon replaced it with another disc that did the same thing, which helped me decide to get the movie (I would have preferred to have a vehicle for hearing the soundtracks that would work in a PC, or a CD player). I am not qualified (or sensitive enough) to 'rate' the movie. I will say if you have never seen it, rent it and fast forward (treat yourself) to the last 2 minutes (see [...] Play it back with no music, then with music. Play it back at different times and through different moods and you will appreciate one of the finest works of film in existence coupled with one of the finest pieces of music ever produced. No single 'scene' can show you the promises of the past and let you feel the horror of regret. A mystical view that soon reveals Anna walking toward the camera, through the tree lined road, passing Holly is a statement and genius unto itself. The Karas compositions are brilliant, but this one is an everyman's masterpiece. What a gift (of course see the film. It is after all - superb!). | Correct Model: Baseline
Review: Oddly strange little book that somehow manages to be quite moving on its own terms.  As much (maybe even more) about Todd Slaughter's somewhat magical life than his Elvis connections.  He writes of his childhood and beyond in clear-eyed, just the facts, ma'am fashion, and the Elvis stories he reveals serve to bring a different, less legendary Elvis to the forefront.  This very human Elvis is much put-upon, manipulated, and taken advantage of; not just by the usual (and genuine) villain, Colonel Tom Parker, but by just about all the suits he has to deal with (from RCA, MGM, music publishers, booking agents, etc.).  At times, even his father and members of the Memphis Mafia are suspect.  All told with compassion and a genuine sense of affection by Todd Slaughter.  Truly, it's a wonder that Elvis didn't flame out even sooner, going by Slaughter's recollections.  The usual self-published boondoggles (poor editing, etc.) apply.  Still, a good read all-in-all. | Correct Model: Baseline
Review: I ordered this with my friend's severely autistic son in mind. This very thin book introduces the concept of five love languages that can be used to "love" your spouse and your child or children with disabilities. <br /><br />The five languages are:<br />- Words of Affirmation<br />- Quality Time<br />- Gifts (or "Receiving Gifts")<br />- Acts of Service<br />- Pysical Touch<br /><br />Two helpful child development categories ( attachment and cognitive development) are also introduced because it is helpful to know where your child stands with regard to these categories. For children with unhealthy attachments, understanding their type of attachment helps the parent to implement these love languages to provide healing.<br /><br />These love languages are used between the parents and also between parent and child. For example with regard to physical touch, one father would rub the cheek of his child when in the NICU to provide physical contact and express love. When that disabled child began to talk, his son would beg to have his cheek rubbed as it would soothe him and let him know he felt loved.<br /><br />In summary, this is an resource for parents with disabled children. The entire premise of this text is to introduce these five love languages and then provide parents with examples of how others implemented them with their children and sometimes with each other. At the beginning there are tests the parent can take to determine their love language. Those tests are also available online.<br /><br />I believe this book is not only good for those who have disabled children but for any adult in a relationship or with a child. However, I did want more. I would have liked five full case studies on each of these love languages in addition to the many unrelated examples. | Correct Model: Final
Review: This hurts.  I've admired Dr. Longo's ingenuity, creativity and genius for a while now.  Loved one of his documentaries on FireStick...watched podcasts and interviews.  Loved each and every single one of them.<br /><br />Then I bought this book.  It all fell appart for me.  I'd have returned it if it wasn't because I had earmarked a few pages.<br /><br />First few parts of the book are mostly autobiographical.  Call me crazy but I was not that interested in that portion of the book.  I mean, he's lead a very rich life, I just don't want to pay to hear about it.  Tell me all about it over a cup of coffee or a glass of wine, for free.<br /><br />The rest of the book is a huge sale on his Prolon FMD product.  Which is fine.  I'd would have loved to know what I was getting myself into before actually paying to buy advertising.<br /><br />IF, and that is a big IF, you are not familiar at all with fasting and its benefits it will serve you better to watch his documentary.  Also, there is another great documentary about fasting, etc. called The Science of Fasting.  Both are great.<br /><br />IF you are unable to fast for health or weight loss due to health issues or social issues or whatever, then sure, you can pay to get the FMD Prolon info from his book ...or...you could get it for free on his website.<br /><br />Hope this helps someone not waste their hard earned pesos. | Correct Model: Baseline
Review: This novel is certainly light reading and full of cliche phrases that made me laugh. I listened to it on Audible. It was actually fun to listen to when I was ill recently; it was a very good stress-reducer. I would hardly call it good writing. The characters are rather flat, corny, and silly. This is a book for teenagers, or possibly, young adults with limited experience of history or good writing. Obviously, Sarah Woodbury borrowed some ideas from the Outlander series, but she does not write nearly as well as Diana Gabaldon.<br /><br />Okay, I changed this from a one star to two stars, since it is rather fun. Young teens would like this book. | Correct Model: Final
Review: Tried four recipes so far- loved them all. Just wish there were more photos in the book. | Correct Model: Baseline
Review: Thought this might help my 24 year old, but this is for babies | Correct Model: Baseline
Review: Not happy with the pictures.... hope it will work.... | Correct Model: Baseline
Review: Not a bad book to learn basic weather stuff. | Correct Model: Baseline
Review: Although dated (1971), Philip J. Klass's SECRET SENTRIES IN SPACE is a good introduction to U.S. (and Soviet) space reconnaissance systems. While I had heard about this book a few years back, what peaked my interest and led me to purchase it was the fact it is mentioned in a recently declassified (September 17, 2011) Top Secret National Reconnaissance Office (NRO) publication, A HISTORY OF THE HEXAGON PROGRAM (April 2012), available on Amazon.com. Of note is this paragraph:<br /><br />"One of the magazines continually publishing articles on the progress of this nation's reconnaissance programs is AVIATION WEEK [or AVIATION LEAK, as it's jokingly referred to it!] Other publications have sporadically covered this area of government activity, but the most prolific writer on the subject has been AVIATION WEEK'S Philip J. Klass. He did a thorough job in summarizing his knowledge of the United States reconnaissance program in a book titled 'Secret Sentries in Space'. Although lacking in some details, and slightly inaccurate in others, little is left to the imagination of our nation's adversaries. What is really damaging is the impact these public revelations have on people involved in reconnaissance programs. After each public announcement of the newest advances in this country's space reconnaissance activities it is necessary for program management in both government and industry to caution their personnel not to discuss or confirm these stories."<br /><br />Enough said! | Correct Model: Final
Review: PISTOLS AND PETTICOATS is kind of an odd duck--an historical survey of female detectives both real (e.g., Kate Warne of Pinkerton fame) and fictional (e.g., Dorothy Sayers' Harriet Vane; Sue Grafton's Kinsey Millhone), and of  female detective-story writers (e.g., Anna Katherine Green, the mother of detective fiction).  The survey is presented more or less in chronological order, so that factual biographies and fictional plot summaries are kind of jumbled together in the chapters.<br /><br />The text is thoroughly researched, and the catalog of female detectives seems complete--at least I couldn't think of a famous female detective (or female detective-story writer) who wasn't covered.  The text itself is very readable.  However, I quickly tired of reading plot summaries (especially when I'd read the novels already), and of working to separate the stories of the genuine female detectives from those of the detective-story authors and of the fictional detectives.<br /><br />To me, PISTOLS AND PETTICOATS seems like the end product of a lot of research that the author decided to publish so that the work wouldn't go to waste.  The book is footnoted, and has some great photographs of the real detectives and detective-fiction writers.  Overall it's a good job, and I think it could be used as a textbook in appropriate college courses (e.g., women's studies, contemporary culture, writing detective fiction).  Also, mystery fans interested in female detectives may be able to use it to find some detectives they've overlooked.  However, because PISTOLS AND PETTICOATS didn't contain any real revelations for me, I rate it at 4 stars (&#34;I like it&#34; on the official Amazon scale). | Correct Model: Baseline
Review: I found this book difficult to keep straight. There are several main characters and there's a lot of jumping around from character to character. The Muslim boy was my favorite character. He was the only character that was approachable, genuine and had good intentions. The whole story takes place over approximately one day, so it was slow moving. It requires a lot of descriptions to cover such a short time span, which the author does pretty well. The beginning of the school day, each character has a secret that doesn't get revealed until the end, so it appears that everyone is suspicious. The actual motives behind the bombing were a little unrealistic in my opinion, however, the topic is timely. Overall, it was interesting and entertaining. | Correct Model: Baseline
Review: Sometimes you have to make something very quickly for a gift, but you don't want it to look like it isn't special.  This book really fit the need for me and I will definitely use it over and over. | Correct Model: Baseline
Review: A disappointment in that I did not enjoy it as much as the author's first novel &#34;Child 44&#34;.  But still an interesting read. | Correct Model: Baseline
Review: Cute book but wish it would come in board book form. | Correct Model: Final
Review: Peter Bowler presents a synopsis of Charles Darwin's contributions to science, history, and culture. This book tries to provide a quick summary of the important periods in Darwin's life, touching briefly on each significant aspect.<br /> Much of the book is written in a somewhat technical way and is a bit too wordy. I had a difficult time maintaining my interest while I was reading some of the chapters. Certain areas deserved more coverage, like the reaction when Darwin went public with his theories.<br /> On the positive side, this book does give some good insight on Darwin's relationships with the other prominent scientists of his time and there are some moments where the slowness of the book becomes more interesting, like the section that covers Darwin's voyage of discovery aboard the Beagle. Overall, however, Bowler does not really present anything new or profound that we haven't heard before. | Correct Model: Final
Review: Simple instructions and so many unusual quilt blocks to choose from.  This book will not disappoint! | Correct Model: Baseline
Review: When I first opened this planner, I LOVED all of the months that are in it! I don’t need the individual days of the month so this worked perfect! But, the dang thing wouldn’t stay open for me to write in it! So I went to FedEx and had them cut off the spine and bind it for me! Do not regret doing that! Otherwise, this is great! | Correct Model: Baseline
Review: Definitely not a great &#34;feel good&#34; experience - most of these stories end far from the garden path or glorious sunset.  There is no doubt that they are well-written - beautifully written actually.  If you enjoy reading a collection of short stories just to appreciate the writing style, this may be for you.  What drives me crazy about a lot of short stories, is the way they dwindle off, leaving the reader wondering what in heck they are supposed to make of the ending.  You'll find that here.<br />Sorry - I appreciate good writing, but there have to be fleshed-out characters, well-described settings, and a story that grabs me and holds on.  I didn't find that here. | Correct Model: Final
Review: KNITTED TEDDY BEARS: DRESSED WITH HANDKNITS has simple patterns for small (10"), medium (13"), and large (17") bears.  I purchased this book because I thought the clothes (including undergarments!) were cute.  Nearly all of the clothing patterns are modeled in the cover photograph and the "Look Inside This Book" feature, which show the jacket, coat, overalls, sweater (striped version), and pants.  In addition, the book has patterns for an undershirt, boxer shorts, a scarf, a charming lacy dress, and a delightful pants suit with a lacy tunic.<br /><br />The patterns are written in row-by-row instructions.  This means that if you use the same yarn and needles for the basic bear and the clothing, the clothing will be sure to fit.  The bears are knit on double-pointed needles:  for the large bear, size US 8; for the medium bear, size US 4; for the small bears, sizes US 1.5 or US 2.5.  For the yarns used in the models, the brand names are given, but not the specific yarn gauges.  You can substitute any yarn that states on the label that it is suitable for knitting on the needle size you use.  The patterns recommend stuffing the bears with wool (but I would at least try stuffing mine with polyester fiberfill).<br /><br />This is quite a slender volume:  just 68 pages (including two pages for "Notes").  Because the book is so short, and has such a limited number of clothing patterns, I rate it at 4 stars ("I like it" on the official Amazon scale). | Correct Model: Baseline
Review: My children really enjoyed this book, and the other Fly Guy books, for that matter. They are not reading yet, so I cannot say that it is a particularly good reader. But for pre-readers, I particularly liked that the kids got a chance to explore adjectives. Fly Guy was looking for something smelly, brown, lumpy, and oozing. They really got into helping describe these words. Yeah, it is kind of gross, but it is a fun gross<br />I'm not a fan of dividing a picture book up as a chapter book. It seems silly and inhibits the flow. | Correct Model: Baseline
Review: This is one of the best SPAM cookbooks I've ever seen. Lots of beautiful pictures, suggestions on how to use SPAM.  Honestly, I am not a huge SPAM fan, but my husband and Mom are.  So I am trying. The cookbook has a spiral back and flaps that you can use to save your place in the cookbook.  There are some silly &#34;helpful&#34; cooking hints along the way, but that doesn't detract from the recipes and beautiful pictures! | Correct Model: Baseline
Review: I liked the message of this book although I must say it was lost on a 6 yr old. I did keep reading it to her so hopefully it sunk in. The only problem with the coloring book was how very few pages were in it. I don't think it was worth the money. She was finished with it very quickly. | Correct Model: Final
Review: Not good as though it would but good | Correct Model: Baseline
Review: Had I the power of foresight, had I known what I was buying, I would have kept my money in my pocket.<br /><br />Still: easy read and for those who have no idea of the "malinvestment" at cyclic tops, get it. There is some math addressing return on land at tops where high land cost must be recouped by building (and financing) extra rentable floors. I liked the simplicity of the thesis. So some truth there...<br /><br />And title says "Prediction" whereas author says: This not a prediction.<br /><br />Best not buy this book as there are much better ways to use precious reading time. Try Rothbard. | Correct Model: Baseline
Review: I love buying from amazon. I am usually a happy customer. I ordered this book because I love the shawshank redemption and wanted to read it again. I ordered the hard cover cause I love to collect good hardcovers (especially Stephen king ones) what I got was a hardcover book that was the size of a soft cover book. when I looked back it does give demensions, but in all my years there have been large hard covers and then small soft covers. I should have know better but I just thought I got a nice deal. im gonna rethink my book buying. I love to buy and own books, but I literally live a block from a library, and with all the ways there are to purchase books now, I think ill wait to see it in front of me. im gonna order less now cause I wanna see it in front of me first now. sad. | Correct Model: Baseline
Review: First of all, some readers have placed this in the "fantasy" genre, which makes no sense to me. It is middle grade fiction, best described as "medieval," I guess, because of Kings and kingdoms and such, but there are really no elements I would associate with "fantasy" (magic, potions, mythical beasts, etc).<br />Second, the only reason this isn't a 5-star review is because I could see the slow, deliberate pace of the book being a drawback for many readers, even though I loved it. It's not necessarily "thrilling" like some reviews say, but I did find it very satisfying at every turn.<br />I found the writing tight, the dialogue clever, the characters complicated and enjoyable. There is a bit of violence and definitely some intrigue, but ZERO language or sexual content, with barely even a hint at possible romance. This book is about the STORY, and I think that's great.<br />Perhaps the only drawback for me was that some of the secrets and intrigue got a little jumbled up at the end and I found it hard to follow exactly what was happening. Sort of like the high points of a James Bond movie - too many intersecting plots and secrets make it a bit confusing. But this didn't override the fact that I really liked this book. I look forward to the rest of the series! | Correct Model: Baseline
Review: "It's a book!" "It's a game!"<br /><br />There is no story in this book. Just random pages with phrases written with numbers in the words. It takes some creative thinking to figure some of them out. I like this book as a bit of a brain teaser for the older elementary crowd. It is over the heads of early readers and makes a completely lousy read aloud. But an older elementary child can read the book and then think of a few wumbers of his or her own. | Correct Model: Final
Review: My android was newer than the sample in the book, but was similar enough to get me started using all the features of my phone.  Great book, just make sure it is compatible with your model phone. | Correct Model: Baseline
Review: My daughter is an avid reader and devours every book I give her.  She said she couldn't even finish this because it was too boring. | Correct Model: Final
Review: I ordered this book as I was involved with operation Hickory.  I was only in country two months when my regiment was assigned duty along the DMZ.  At the young age of 19, I had no knowledge of what happened during my deployment in the operation. | Correct Model: Final
Review: All information is readily available online for free. | Correct Model: Baseline
Review: Enjoyed reading this mystery.  The main characters were interesting and reasonably well rounded.  The crime was awful and one of a series of a murders.  Reasonable well plotted.  Would read more about these characters. | Correct Model: Baseline
Review: Childlike | Correct Model: Final
Review: This book covers several types of dementias (Alzheimer's disease, Lewy-body dementia, vascular dementia, & frontotemporal dementia); however, it predominately concentrates on Alzheimer's. Each chapter is designed to stand on its own but I read this cover to cover and find it is comprehensive. It talks about dementia tests, drugs, what scams not to fall for, day-to-day care, reality orientation and validation therapy for those with Alzheimer's, what to look for in a caregiver, etc. It focuses more on the caregiver side and soft skills associated with dementia and less on the scientific aspect of the disease.<br /><br />This is a comprehensive text but is not the only text that is needed in my opinion, especially if one has a dementia other than Alzheimer's. For instance, I have a friend who I suspect has Parkinson's with Dementia (PDD) of the Lewy body kind and I feel [[ASIN:193260393X A Caregiver's Guide to Lewy Body Dementia]] is a much better resource for that disease.<br /><br />In my opinion, this is a great Alzheimer's for dummies book and a good demenia (of any other kind) for dummies book. | Correct Model: Baseline
Review: Bought for daughter’s required summer reading. Very quick to arrive! | Correct Model: Baseline
Review: I thought this was going to give me a couple of things history put on paper a little an entirely different way.  Lots of just black and whit on a page with some inked pages of drawings.  I had a coule other books to try to get me to the cutting edge stuff which is what I was needing at the time and it still sits on my shelf opened one time...look into its contents before buying I thought a different outcome from the title. | Correct Model: Baseline
Review: Not my favorite and the kids were confused. I get it though. | Correct Model: Baseline
Review: Hurricane Sandy made landfall in New Jersey in the fall of 2012 and Money Island and Bay Point located in the Bayshore area of Cumberland County, New Jersey were never rebuilt. Instead they are like ghost towns with only a resident or two remaining as of 2019.<br /><br />I never heard of these shore towns on the New Jersey side of the Delaware Bay. I also thought it was mostly the ocean-side communities that were hardest hit. This author grew up in the Bayshore area and then left and came back to write this story. These towns were forgotten during Sandy's rebuild mostly due to the fact that it did not make economic sense to put money where tourism dollars were unlikely to be spent. Instead many homeowners sold out to the Blue Acres program that bought up the properties to turn it back into nature.<br /><br />What I found very disconcerting was that the Health Department in Cumberland County started issuing violations to Sandy victims for septic failures that were either the result of years of decay or Sandy's storm damage. How can these poor victims pay for daily fines and fix issues when they do not even have the money to rebuild? We can always depend on the government to destroy a person further when victims are homeless due to no fault of their own. Although, I can understand that the cost benefit analysis of investing in infrastructure for "ghost towns" does not make economic sense, it also does not make sense that these towns were not even known by the bureaucrats when allocating funds. One has to be aware of a problem before it can be fixed.<br /><br />In summary, I found this story to be very interesting and also sad. I am amazed at the fortitude of the couple of homeowners who remain behind and did not sell out (particularly Mike and Kate). | Correct Model: Final
Review: I'm struggling a bit to get into it, but will keep trying! | Correct Model: Baseline
Review: while I absolutely adore almost everything James Patterson writes, I'm not a fan of his on this particular storyline. It's just me and I'm sure almost everyone else will find it riveting. | Correct Model: Final
Review: This book was a little difficult to get into. Once I was about half way through, the paced picked up. The characters weren't particularly likeable. The main character was shallow and although she does go through a transformation, I still wasn't crazy about her. There is a lot of sex and alcohol in the book. I don't live in a cave - I know that's out there, but as a mom to a teenage daughter, I was hyper aware of it. For the most part,  it was entertaining and I enjoyed the fabulous trip they went on.  I don't have a problem letting my daughter read this. | Correct Model: Baseline
Review: This is a very interesting book about the author's experience with birds, many of which she rehabilitated or raised from fledglings. The author is also a good sketcher and sketches many of the birds she handled peppering the book with many of these delightful images. Sometimes she temporarily "borrows" wild birds from their nest to sketch. This is not something I agree with but this practice seems to have litte affect on the birds according to the way she justifies it.<br /><br />It is obvious that Julie has a dislike of free-roaming cats who injure birds and several of the birds she rehabilitated or attempted to rehabilitate were mauled by cats.<br /><br />She talks about phoebes, starlings, chimney swifts, a titmouse, grouse, etc., etc.  She also included a very interesting story about a turkey vulture which throws up in her car.  Apparently turkey vulture vomit is awfully fetid--not that other vomit isn't but this is worse.  I like how she ends her book with Charlie, her parrot.  I did not realize parrots were such bad pets.  She defines a pet as an animal whose emotional needs can be met by a human being and a parrot does not fit this definition according to Julie.<br /><br />Her sketches throughtout the book are wonderful and her bird stories kept my interest.  She named many of the birds and when rehabilitated always released them if they could live on their own.  Julie is definitely a "bird mother" and from my perspective her husband Bill is a saint to put up with it all.  Although I'm not a birder, it was great to read about Julie's experiences and I'm am thankful their exists all forms of people in this world who take care of orphaned animals avian or otherwise. | Correct Model: Baseline
Review: I am a fan of historic novels--many of them either medieval or renaissance mysteries, some other forms of historic novel.  This is one of the best series I've read yet, well-researched and so much more than just a 'murder mystery'.  They give a good appreciation of specific times and events during the later years of King Henry VIII's reign, provoking a desire to learn more about the times (even though I've read a lot about the Tudors in factual detail) -- such as a biography of Thomas Cromwell, whom I'd never thought to be interested in before, as well as the dissolution of the monasteries.  Henry, himself, is not painted in a sympathetic light in any of the novels, which I believe, for this period, is quite accurate.  As a young king, Henry was much more admirable, but became increasingly self-serving, avaricious and disinterested in his people beyond how they could contribute to his aspirations to conquer and his greed for luxury. | Correct Model: Baseline
Review: For an O'Reilly book, DESIGNING FOR WEARABLES: EFFECTIVE UX FOR CURRENT AND FUTURE DEVICES seems a little short on technical content.  About half of it is a fairly comprehensive, descriptive catalog of existing wearable devices (e.g., FitBit, Jawbone, Pebble smartwatch, Google Glass, Lumo Lift, GoPro, Pokemon Go, Thync).  The remaining half is devoted to to actual designer's thinking, tools, problems, and methods (e.g., service design and machine learning, embodiment and perception, prototyping and prototyping tools, selling the invisible).<br /><br />The author points out that wearable devices offer &#34;services&#34;, so that designers must focus on UX (user experience), and create interfaces that translate the data gathered by the devices into things that are helpful and meaningful to the users.  For example, an activity tracker that records steps is a mere pedometer, unless the tracker does extras, such as tabulating the wearer's daily, weekly, and monthly activity, heart rate, calories burned, sleep time, etc.<br /><br />For a wearables designer, an important consideration is figuring out what sort of technology is available to gather the data that can be reported in ways that the user will find helpful and motivating.  For example, the Fitbit Charge HR uses a photoplethysmogram (PPG) sensor to measure heart activity, and a gyroscope and accelerometer to count steps.<br /><br />If you're not an actual designer, but happen to own one (or more) of these devices, it's interesting just to read up on what it's intended to do (from a designer's perspective), and how it actually works. | Correct Model: Final
Review: This coloring book has a lot of cute animal pages. Some of the pages are heavier on the designs and others have less. I prefer the ones that are more design when I color. The others have cutesy animals with full animal shapes to color in which make it more child friendly. The paper quality is average and may show minimal bleed through depending on the markers you use. The pages are perforated and tear out fairly easily. | Correct Model: Final
Review: Cute book not sure if the flaps will hold up. | Correct Model: Baseline
Review: This book was intensely HOT. The author has a wonderfully large vocabulary. I had to look up words fairly often; some of which weren't even in Wikipedia :) However, given these positive opinions, the author's grammar leaves much to be desired, particularly her inept use of pronouns. I know editing for grammar is grunt work but it is part of the craft. For me it made the difference between four and five stars. | Correct Model: Baseline
Review: this is a great compendulum of classic indian recipes, with great photos, descriptions of the regions and food.  some hard ingredients to locate and maybe its my cooking (although I am an avid cook) - the end result was rather bland.  Internet recipes seem to work better.  I was hoping for great experiences and felt it was lacking in the end. | Correct Model: Final
Review: my x bought this with out my knowledge and i cant stand people who dont have the capacity to have their own imagination so they have to read someone elses,.,.,. | Correct Model: Baseline
Review: I love Kristan Higgins books and this one did not disappoint. Let me start by saying that I read for fun. I want to be entertained. I am not nit picky about the depth of characters or details about the plot. I agree with some of the other reviewers that it started out slowly. But after the first few chapters it does pick up and I had difficulties putting it down. I stayed up late just to finish it. I found the characters to be likeable. That's important to me. The chapters alternate between the two sisters' point of views. I started out liking Jenny better than Rachel, but by the end, I grew to like Rachel, too. I follow the author on FB and the book sounds just like her! Higgins is quirky and funny and so was this book. Yes, there's a little bit of language, but it's not excessive by any means. I think it just makes it more realistic. Both sisters go through some personal growth and I was happy for them both in the end. I purchased this book months ago and I am sorry that it took me so long to find the time to read it. | Correct Model: Baseline
Review: Can't wait to try these patterns! | Correct Model: Baseline
Review: Very technical. | Correct Model: Baseline
Review: I purchased both the ([[ASIN:0981994644 More Chat Pack Cards: New Questions to Spark Fun Conversations]] and the [[ASIN:0975580167 Chat Pack: Fun Questions to Spark Conversations]] (this set)  to add variety to my vacation time with my boyfriend. Both he and I liked how these cards stimulated conversations and allowed us to think and get to know details about each other that we did not know even though we have been together for 15 years.<br /><br />Each card is the size of a tiny Post-It note at 1 inch x 3-1/2 inches. In this package are questions such as:<br /><br />- Thinking back to all the great TV series finales that you have seen over the years, which show do you believe had the best final episode?<br />- If snow could fall in any flavor, what flavor would you choose?<br />- Which age, when you turned it, was the most difficult for you to accept?<br /><br />Some questions made us think on the fly and make stuff up. This means that if we were answering this question again, we would give a different answer because we never actually thought about that particular question. There are some questions that were difficult to answer. For instance, I do not watch TV or movies much so I could not answer any questions related to the best movie sequel because I did not know any. One is encouraged to change up the questions if necessary.<br /><br />Overall, we found this both fun and entertaining and only got though part of each pack. We're saving the rest of the pack for the next vacation or time together. | Correct Model: Baseline
Review: After Boing Boing featured this book twice I had to see what the excitement was about....<br /><br />I loved this book and could not put it down. | Correct Model: Baseline
Review: Same book as Humble Pie | Correct Model: Final
Review: Was just curious and wanted a bunch of info that was easy to read.  Check, check.  In my opinion this book is great collection of information.  If you are just looking into options, this book is great place to start.  I loved that it was really easy to read - not a lot of excess - just down flat info.  Even though I don't think this is a road we are going to go, it was well worth the investment to look into the options. | Correct Model: Baseline
Review: It's a nice map but it's old. I didn't notice that it was 5 years old when I ordered it, my bad.  I figured it out right away, however, as the road between Puerto Penasco and Golfo de Santa Clara is missing. :-( I like the look of it and the paper is sturdy and tear proof, but it's in dire need of an update. As it is, it's not really worth it. | Correct Model: Baseline
Review: Such a cute sticker miniature book! Filled with lots of little hedgehogs. For $1.99 I have no complaints. Would buy it again | Correct Model: Baseline
Review: Is more of a journal - than a cook book. Journal of meals Alice enjoyed. Some recipes but not much. | Correct Model: Final
Review: Prior to reading this, I was unfamiliar with the deadly London fog or Reg Christie. I read The Devil in the White City many years ago and loved it, so since this was compared to it, I wanted to give it a try. The subject was interesting and well researched. It was a bit slow and choppy at times. I felt that it jumped around a lot to the point that it confused me every now and then. It didn't have the flow that The Devil in the White City by Erik Larson. I don't usually read nonfiction books, but Larson's book read like a novel. I didn't feel that I was reading nonfiction. This one didn't read as easily. There were a few times where it resembled fiction, but most of the time it felt like I was reading something for school. Overall, I did like it. It was informative and interesting. | Correct Model: Baseline
Review: I love getting inspiring cookbooks but I feel out of my element with this one.  I guess I don't know Mediterranian cuisine well enough to be able to picture things.  We flipped through it looking for inspiration for supper last night and ended up googling dishes so we could get an idea about what they were.  In the end we made a Asparagus Risotto because we had everything we needed on hand.  But even that we ended up googling so we had an image to aim for.<br /><br />The book appears to have a lot of good ideas and I'm sure we will put some into our normal rotation as they seem simple yet different.  If it had been better illustrated, it would have been exactly what we were hoping for. | Correct Model: Final
Review: This was an interesting book although not going to be a family classic.  The poetry was ok, as was the story.  Straightforward, uncomplicated and not overly literary.  It is a quick read and although I did not find it to be the best, I do think this would make an absolutely wonderful intro to poetry writing.  Each poem has a title and a subtitle that tells you what kind of poetry is being used in the poem.  You can then reference the back which tells more about the type of poetry.  I think it would be an invaluable reference for early year poetry classes.<br /><br />The art is cartoony and good and supports the narrative poetry.  All in all a decent book, but not great. | Correct Model: Baseline
Review: Glad I didn't pay to go see the movie!  Thank goodness it was an audio book or I would never have finished it! | Correct Model: Baseline"""

# Split the result into individual reviews
reviews = mismatched_reviews.strip().split("\n")

# Lists to store reviews based on the correct model
baseline_correct_reviews = []
final_correct_reviews = []

# Process each review and categorize based on the 'Correct Model'
for review in reviews:
    review_text, correct_model = review.split(" | Correct Model: ")
    if correct_model == "Baseline":
        baseline_correct_reviews.append(review_text.strip())
    elif correct_model == "Final":
        final_correct_reviews.append(review_text.strip())

# Output the categorized reviews
print("Reviews where Baseline Model was Correct:")
for review in baseline_correct_reviews:
    print(review)

print("\nReviews where Final Model was Correct:")
for review in final_correct_reviews:
    print(review)

Reviews where Baseline Model was Correct:
Review: A chaotic tornado of characters.
Review: I know Dan Jenkins is a coveted sports writer but man, can he ease up on the "n" word. I'm a golfer but I was turned off by it. Watch the HBO movie version. They substitute the "n" word with "ethnics" which is (sigh) obvious. In my opinion, Randy Quaid has the best golf swing for an actor. And there's a good nude scene by Corinne Bohrer.
Review: Purchased for my husband for Christmas.... he has always wanted to build a wooden boat.  He loves that this one has some actual plans that are good to consider as a beginner.  He notes that you'll probably need a couple of different books to compare and gather info, but this one is an excellent one to start with.  I would consider that a 5 star review from him. I always update if there is anything important to report.  I will say the book arrived with a folded corner and not &#34;packed&#34; in any way, along with other items I had ordered.  That part was

In [34]:
weights = {}
for i in range(12,13):
  test_scalar = get_scalar_mixing_model(i,1)
  train_model(test_scalar, 10, dataloader, train_data, learning_rate = 0.1)
  print(i, ":", test_scalar.layer_weights)
  weights[i] = test_scalar.layer_weights

tensor(0.7202, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


KeyboardInterrupt: 

In [ ]:
lrs = [0.1, 0.01, 0.001, 0.0001]
for lr in lrs:
  for i in range(1,5):
    test_scalar = get_scalar_mixing_model(2, i)
    train_model(test_scalar, 10, dataloader, train_data, learning_rate = lr)
    print("Learning rate:", lr, "Number of layers", i)

## Analysis

In [ ]:
summary(baseline_model)

In [ ]:
summary( get_custom_bert_model(10))